
Team Members

*   Libin Kutty - Group 2
*   Ritu Gahir - Group 3
*   Viju Sudhi - Group 10

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
print(os.getcwd())
os.chdir("/content/drive/My Drive/Colab Notebook/IDL/")
print(os.getcwd())

/content
/content/drive/My Drive/Colab Notebook/IDL


In [5]:
!python prepare_data.py shakespeare_input.txt skp

2020-05-26 06:43:07.661102: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

In [6]:
from prepare_data import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'p': 1, 'O': 2, '-': 3, 'h': 4, ' ': 5, 'Q': 6, ',': 7, 'G': 8, 'f': 9, 'C': 10, 'J': 11, 'w': 12, '3': 13, 'k': 14, '$': 15, ':': 16, 'l': 17, 'P': 18, 's': 19, 'r': 20, 'o': 21, ']': 22, 'x': 23, 'v': 24, 'B': 25, '?': 26, 'b': 27, 'V': 28, 'c': 29, 'g': 30, '.': 31, '&': 32, 'A': 33, 'I': 34, 'X': 35, 't': 36, 'F': 37, 'H': 38, 'i': 39, 'u': 40, 'M': 41, 'L': 42, 'E': 43, 'D': 44, 'U': 45, 'R': 46, '[': 47, 'K': 48, 'n': 49, 'd': 50, 'W': 51, 'Z': 52, 'a': 53, 'z': 54, '\n': 55, ';': 56, "'": 57, 'm': 58, 'T': 59, 'y': 60, 'q': 61, 'j': 62, 'e': 63, 'Y': 64, 'N': 65, 'S': 66, '!': 67, '<S>': 0}
68


In [0]:
@tf.function
def loss_fn(logits, labels):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

In [0]:
class RNN():
  def __init__(self, hidden_size, vocab_size):
    
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size
    self.Wxh = tf.Variable(tf.random.uniform(shape=[vocab_size,hidden_size], minval=-0.01, maxval=0.01, dtype=np.float32)) # input to hidden
    self.Whh = tf.Variable(tf.random.uniform(shape=[hidden_size,hidden_size], minval=-0.01, maxval=0.01, dtype=np.float32)) # hidden to hidden
    self.Why = tf.Variable(tf.random.uniform(shape=[hidden_size, vocab_size], minval=-0.01, maxval=0.01, dtype=np.float32)) # hidden to output
    self.bh = tf.Variable(np.zeros((1, hidden_size), dtype=np.float32)) # hidden bias
    self.by = tf.Variable(np.zeros((1, vocab_size), dtype=np.float32)) # output bias
    self.prev_h = tf.Variable(np.zeros((200, hidden_size), dtype=np.float32)) #Initial hidden output
    self.loss = 0.0
    self.opt = tf.optimizers.Adam()
  
  #@tf.function
  def train(self, inputs):

    #tf.TensorArray(tf.float32, size=0, dynamic_size=True)
    n_time_step = 128 #len(inputs)

    loss_ts = tf.TensorArray(tf.float32, size=128, dynamic_size=True)
    #loss_ts = []
    self.prev_h = tf.Variable(np.zeros((128, hidden_size), dtype=np.float32))
    ts = 0

    with tf.GradientTape() as t:

      for i in tf.range(n_time_step):

        #for inp in inputs:
#        for seq in range(sequence-1):

        inp_curr = inputs[:,i]
        inp_next = inputs[:,i + 1]

        inp_curr = tf.one_hot(inp_curr,vocab_size)   
        inp_next = tf.one_hot(inp_next,vocab_size) 

        # inp_curr = np.array(inp[i][seq])
        # inp_curr = inp_tmp.reshape(1, vocab_size)
        # nxt = seq + 1
        # inp_next = np.array(inp[i][nxt])
        # inp_next = inp_nxt.reshape(1, vocab_size)

        self.prev_h = tf.matmul(inp_curr, self.Wxh) + tf.matmul(self.prev_h, self.Whh) + self.bh
        output = tf.matmul(self.prev_h, self.Why) + self.by
#          output = tf.nn.softmax(output)

        self.loss = loss_fn(output, inp_next)
        loss_ts = loss_ts.write(ts, self.loss)
        #loss_ts = loss_ts.write(i, self.loss)
        ts +=1
        #loss_ts.append(loss_temp)

      loss_ts = loss_ts.stack()
      self.loss = tf.reduce_mean(loss_ts)
    #loss_temp = loss_temp/count

    grad = t.gradient(self.loss, [self.Why, self.by, self.Whh, self.bh, self.Wxh])
    grad, _ = tf.clip_by_global_norm(grad,1)
    self.opt.apply_gradients(grads_and_vars=zip(grad, [self.Why, self.by, self.Whh, self.bh, self.Wxh]))


In [71]:
hidden_size = 512
n_time_step = 128
model = RNN(hidden_size=hidden_size, vocab_size=vocab_size)

#inputs = tf.one_hot(next(iter(data.batch(128))), vocab_size) #, axis=-1
#data_temp = data.batch(128).repeat(10)
for step in range(3000):
  inputs = next(iter(data.batch(128)))
  model.train(inputs)
  if not step % 100:
    print('Step %2d: loss=%2.5f' %
      (step, model.loss))

Step  0: loss=4.21951
Step 100: loss=2.63523
Step 200: loss=2.18399
Step 300: loss=2.00553
Step 400: loss=1.89890
Step 500: loss=1.81429
Step 600: loss=1.75012
Step 700: loss=1.67298
Step 800: loss=1.60284
Step 900: loss=1.54685
Step 1000: loss=1.49681
Step 1100: loss=1.44406
Step 1200: loss=1.42836
Step 1300: loss=1.35474
Step 1400: loss=1.33471
Step 1500: loss=1.29277
Step 1600: loss=1.25375
Step 1700: loss=1.20564
Step 1800: loss=1.18329
Step 1900: loss=1.19481
Step 2000: loss=1.12396
Step 2100: loss=1.10734
Step 2200: loss=1.05607
Step 2300: loss=1.07980
Step 2400: loss=1.04818
Step 2500: loss=1.00568
Step 2600: loss=0.98113
Step 2700: loss=0.94653
Step 2800: loss=0.95744
Step 2900: loss=0.88969


In [0]:
def sample(length, model):

  hprev = tf.Variable(np.zeros((1, hidden_size), dtype=np.float32))

  #x = np.zeros((1, vocab_size), dtype=np.float32)
  x = [0]
  
  ixes = []
  for i in range(length):
    input_c = tf.one_hot(x[-1:],depth=model.vocab_size)
    hprev = tf.matmul(input_c, model.Wxh) + tf.matmul(hprev, model.Whh) + model.bh
    y = tf.matmul(hprev, model.Why) + model.by
    output = tf.nn.softmax(y)

    ix = np.random.choice(range(vocab_size), p=np.ravel(output))
    #x = np.zeros((1, vocab_size), dtype=np.float32)
    #x[ix] = 1
    x.append(ix)
    
    ixes.append(ix)

  return ixes

In [124]:
sample_ix = sample(500, model)
txt = ''.join(ind_to_ch[ix] for ix in sample_ix)
print(txt)

 wish the whs,
Thay do dimeno thond? what.

Ar,: To Maveigs moik. :
'Cwaliold ypideis ghon go he! Pratcerny
hee: A wis lvy gema;
They'l hey. Sheagom pbaf Roman'e whim ssicis
Thian patprecusiay wa chiup tirid, O, bry dinge
whil feampelfige powareaf.
Th yous Marsiug breccesim semane wo'l.

BRUTUS:
The piang atw, wore piof edow aScranselp cconde:
Thing hes offelind: Let ius,
Whor
ExcA l'me bavy, big ikele bace.

VOLUMTIS:
TMegSiuTShYe!

MpNwer'sw: Th Suip
OrIN the'd doa. Ye pe: Thbuur?

ALERIAlS:
O
